In [96]:
import numpy as np
import scipy as sp

In [97]:
from collections import namedtuple
import codecs

In [98]:
TEXTFILE = "1251.full.peter-watts-starfish.txt"
N_OBSERVATIONS = 50000

## 1. Читання та обробка тексту
Складність тут полягала у правильному кодуванні\декодуванні. Етапи обробки тексту вийшли наступними
1. Експорт word файлу в plaintext
2. Перекодування вручну в cp1251. Це текстове кодування зручне тим, що підтримує всі потрібні символи українського алфавіту, але при цьому кожен символ займає лише один байт, як в ascii. Таким чином можно зчитати текст просто як послідовність байтів
3. Читання у масив.  
    Тут виконується другий етап фільтрації. Я 
    1. ігнорую всі символи окрім ```'іґєїабвгдежзийклмнопрстуфхцчшщьюя``` та пробіла
    2. перетворюю всі великі літери на малі
    2. з'єдную всі послідовності пробілів так, щоб підряд міг іти лише один пробіл 

У якості матеріалу я взяв єдиний художній текст, що був у мене на комп'ютері: нещодавно виданий український переклад науково-фантастичної книги "Морська зірка" Пітера Уоттса.  

In [99]:
LETTERS = bytes(" 'іґєїабвгдежзийклмнопрстуфхцчшщьюя", '1251')

def notalpha(c):
    return c < 0xBF and c!=0x20 and c!=0xA5 and c!= 0xAA and c!=0x27 and c!=0xAf and c!=0xB2 and c!=0xB3 and c!=0xB4 and c!= 0xBA

def tolower(c):
    # if c < 0 or c > 255: return None
    dict = (0x0, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7, 0x8, 0x9, 0xa, 0xb, 0xc, 0xd, 0xe, 0xf, 0x10, 0x11, 0x12, 0x13, 0x14, 0x15, 0x16, 0x17, 0x18, 0x19, 0x1a, 0x1b, 0x1c, 0x1d, 0x1e, 0x1f, 0x20, 0x21, 0x22, 0x23, 0x24, 0x25, 0x26, 0x27, 0x28, 0x29, 0x2a, 0x2b, 0x2c, 0x2d, 0x2e, 0x2f, 0x30, 0x31, 0x32, 0x33, 0x34, 0x35, 0x36, 0x37, 0x38, 0x39, 0x3a, 0x3b, 0x3c, 0x3d, 0x3e, 0x3f, 0x40, 0x61, 0x62, 0x63, 0x64, 0x65, 0x66, 0x67, 0x68, 0x69, 0x6a, 0x6b, 0x6c, 0x6d, 0x6e, 0x6f, 0x70, 0x71, 0x72, 0x73, 0x74, 0x75, 0x76, 0x77, 0x78, 0x79, 0x7a, 0x5b, 0x5c, 0x5d, 0x5e, 0x5f, 0x60, 0x61, 0x62, 0x63, 0x64, 0x65, 0x66, 0x67, 0x68, 0x69, 0x6a, 0x6b, 0x6c, 0x6d, 0x6e, 0x6f, 0x70, 0x71, 0x72, 0x73, 0x74, 0x75, 0x76, 0x77, 0x78, 0x79, 0x7a, 0x7b, 0x7c, 0x7d, 0x7e, 0x7f, 0x90, 0x83, 0x82, 0x83, 0x84, 0x85, 0x86, 0x87, 0x88, 0x89, 0x9a, 0x8b, 0x9c, 0x9d, 0x9e, 0x9f, 0x90, 0x91, 0x92, 0x93, 0x94, 0x95, 0x96, 0x97, 0x98, 0x99, 0x9a, 0x9b, 0x9c, 0x9d, 0x9e, 0x9f, 0xa0, 0xa2, 0xa2, 0xbc, 0xa4, 0xb4, 0xa6, 0xa7, 0xb8, 0xa9, 0xba, 0xab, 0xac, 0xad, 0xae, 0xbf, 0xb0, 0xb1, 0xb3, 0xb3, 0xb4, 0xb5, 0xb6, 0xb7, 0xb8, 0xb9, 0xba, 0xbb, 0xbc, 0xbe, 0xbe, 0xbf, 0xe0, 0xe1, 0xe2, 0xe3, 0xe4, 0xe5, 0xe6, 0xe7, 0xe8, 0xe9, 0xea, 0xeb, 0xec, 0xed, 0xee, 0xef, 0xf0, 0xf1, 0xf2, 0xf3, 0xf4, 0xf5, 0xf6, 0xf7, 0xf8, 0xf9, 0xfa, 0xfb, 0xfc, 0xfd, 0xfe, 0xff, 0xe0, 0xe1, 0xe2, 0xe3, 0xe4, 0xe5, 0xe6, 0xe7, 0xe8, 0xe9, 0xea, 0xeb, 0xec, 0xed, 0xee, 0xef, 0xf0, 0xf1, 0xf2, 0xf3, 0xf4, 0xf5, 0xf6, 0xf7, 0xf8, 0xf9, 0xfa, 0xfb, 0xfc, 0xfd, 0xfe, 0xff)
    return dict[c]

def read_textfile(file, char_limit=1000000):
    data = bytearray()
    with open(file, 'rb') as f:
        checkpoint = 10000
        n_res_bytes = 0
        n_read_bytes = 0
        counter = 0

        c = 0x20
        prev_c = ' '
        prev_w = True
        while True:
            n_read_bytes += 1
            d = f.read(1)
            if len(d) == 0:
                c = None
                break
            c = d[0]

            if notalpha(c): 
                # print(f"'{c}' is not alpha")
                c = 0x20
            if c == 0x20:
                # print("space")
                if prev_w: continue
                prev_w = True
            else:
                prev_w = False
            
            c = tolower(c)
            # print(f"tolower {c}")
            data.append(c)
            prev_c = c
            counter += 1
            n_res_bytes += 1
            if counter == checkpoint:
                counter = 0
                print(f"read checkpoint {n_res_bytes} ({n_read_bytes})")
            if n_res_bytes > char_limit: 
                break
        print("read - end")
    return data


In [100]:
notalpha(0x0)

True

In [101]:
def count_freqs(data):
    freqs = dict()
    for c in data:
        n = freqs.setdefault(c, 0)
        freqs[c] = n+1
    return freqs

---

## 2. Навчання ПММ. Алгоритм Баума-Велша

Я не буду повторювати лекцію та описувати як він працює. Натомість опишу проблеми, з якими зіткнувся під час реалізації
- *Правильно записати матричні операції.*  
    Можна було реалізувати їх просто циклами, але матричними операціями я пришвидшив роботу алгоритму в 1.5-2 рази. Проте, оскільки у формулах використовуються складні суміші згорток, це зайняло суттєво багато часу.

- *Правильно ініцаілізувати матричні змінні.*  
    При обчисленні нової оцінки $\hat B$ необхідний ітераційний алгоритм, бо на індекси накладаються складні умови. Я забув заповнити початкове значення $\hat B$ нулями. В результаті матриця обчислювалася неправильно. Примітивна помилка, але забрала в мене пару годин життя. 

- *Розібратися в нотації*  
    В деяких джерелах наша $\gamma_t(i,j)$ називається дігаммою та позначається так само, а в деяких - позначається $\xi_t(i,j)$, при тому що гаммою позначається інша величина.

- *Підібрати доречне початкові значення $\mu, A, B$*  
    Спочатку алгоритм практично не покращував оцінку. Проблема була в тому, що я вибрав у якості початкового значення рівномірні розподіли.

    Після цього для рядків матриць я використовував величини 
    
    $$\mu_i = \frac{1}{N} + (\xi_i - 0.5)*e,\quad a_{ij} = \frac{1}{N} + (\xi_{ij} - 0.5)*e, \quad b_{ij} = \frac{1}{M} + (\xi'_{ij} - 0.5)*e$$
    $$\xi_i, \xi_{ij}, \xi'_{ij} \sim U[0;1], iid.$$

    Значення $e = 0.1$ виявилось занадто великим і навчання моделі не давало результатів за 100 ітерацій.
    Доречним виявилось значення $e = 0.001$.
    Також я взяв з опублікованого розв'язку ідею покласти початкове значення

    $$A = \begin{pmatrix}0.4 & 0.6\\ 0.6 & 0.4\end{pmatrix}$$

### 2.1. Код

In [102]:
BaumWelch_params = namedtuple('BaumWelch_params', 'max_iter, eps, n_hidden_states, n_observation_states, n_observations')
STOCHASTIC_EPS = 1e-8

In [114]:

# X, Y: T
# mu: 1xN
# A: NxN
# B: NxM

# P_lambda(Y0=y0,...,YT=yT) = alpha[T].sum()
def calculate_alpha(Y, estMu, estA, estB, params):
    N, M, T = params.n_hidden_states, params.n_observation_states, params.n_observations
    alpha = np.empty((T, N))
    C = np.empty(T)

    alpha[0] = estMu * estB[:, Y[0]]
    C[0] = 1/alpha[0].sum()
    alpha[0] *= C[0]
    
    for t in range(1, T):
        # alpha[t,k] = sum_i alpha[t-1, i] * A[i, k] * B[k, y[t]]
        alpha[t] = (alpha[t-1] @ estA) * estB[:, Y[t]]
        # for k in range(N):
        #     alpha[t, k] = (alpha[t-1] @ estA[:,k]) * estB[k, Y[t]] # todo: optimize
        #     if np.abs(alt[k]-alpha[t,k]) > 1e-5: raise ValueError(f"alpha different at {t},{k}")
        C[t] = 1/alpha[t].sum()
        alpha[t] *= C[t]
    
    if alpha.shape != (T, N): raise ValueError("ShapeMismatch")
    return alpha, C

# P_lambda(Y0=y0,...,YT=yT) = Mu * beta[0] @ B
def calculate_beta(C, Y, estMu, estA, estB, params):
    N, M, T = params.n_hidden_states, params.n_observation_states, params.n_observations
    beta = np.empty((T, N))

    beta[T-1, :] = C[T-1]

    for t in range(T-2, -1, -1):
        # beta[t,k] = sum_i  beta[t+1, i] * A[k,i] * B[i, Y[t+1]] * c
        beta[t] = (estA * estB[:, Y[t+1]] * beta[t+1]).sum(axis=1) * C[t]
        # for k in range(N):
        #     beta[t,k] = (estA[k] * estB[:, Y[t+1]] * beta[t+1]).sum() * C[t] # todo: optimise
        #     if np.abs(alt[k]-beta[t,k]) > 1e-6: raise ValueError(f"beta different at {t},{k}")

    if beta.shape != (T, N): raise ValueError("ShapeMismatch")
    return beta

def calculate_gamma(alpha, beta, params):
    N, M, T = params.n_hidden_states, params.n_observation_states, params.n_observations
    gamma = np.empty((T, N))
    gamma = alpha * beta
    gamma = gamma / gamma.sum(axis=1, keepdims=True)
    
    if (np.abs(gamma.sum(axis=1) - 1) > STOCHASTIC_EPS).any(): raise ValueError(f"gamma is not stochastic! s={gamma.sum(axis=1)}")
    if gamma.shape != (T, N): raise ValueError("ShapeMismatch")
    return gamma

def calculate_gamma_alt(xi, params):
    N, M, T = params.n_hidden_states, params.n_observation_states, params.n_observations
    gamma = np.empty((T, N))
    gamma = xi.sum(axis=2)
    
    # if (np.abs(gamma.sum(axis=1) - 1) > STOCHASTIC_EPS).any(): raise ValueError(f"gamma is not stochastic! s={gamma.sum(axis=1)}")
    if gamma.shape != (T, N): raise ValueError("ShapeMismatch")
    return gamma

def calculate_xi(alpha, beta, Y, estMu, estA, estB, params):
    N, M, T = params.n_hidden_states, params.n_observation_states, params.n_observations
    xi = np.empty((T, N, N))
    for t in range(0, T-1):
        xi[t] = alpha[t].reshape(-1,1) * estA * estB[:, Y[t+1]] * beta[t+1]
        # xi[t] /= xi[t].sum()
        # if (np.abs(xi[t].sum() - 1) > STOCHASTIC_EPS): raise ValueError(f"xi[{t}] is not stochastic! s={xi[t].sum()}")
    # print("xi sum mean: ", xi.sum(axis=1).mean())
    
    if xi.shape != (T, N, N): raise ValueError("ShapeMismatch")
    return xi

def reestimate(alpha, beta, gamma, xi, Y, params):
    N, M, T = params.n_hidden_states, params.n_observation_states, params.n_observations

    estMu = gamma[0]
    estA = xi.sum(axis=0) / gamma[:-1].sum(axis=0).reshape(-1,1)
    
    estB = np.zeros((N,M))

    for t in range(0, T):
        estB[:,Y[t]] += gamma[t]
    estB = estB / gamma.sum(axis=0).reshape(-1,1)

    return estMu, estA, estB

def evaluate(C):
    # мінус _потрібен_!
    return -np.log(C).sum()

def learn_iterate(Y, Mu, A, B, params, alpha=None, c=None):
    if (alpha is None or c is None):
        alpha, c = calculate_alpha(Y, Mu, A, B, params)
    beta = calculate_beta(c, Y, Mu, A, B, params)
    xi = calculate_xi(alpha, beta, Y, Mu, A, B, params)
    gamma = calculate_gamma(alpha, beta, params)
    # gamma = calculate_gamma_alt(xi, params) 
    return reestimate(alpha, beta, gamma, xi, Y, params)

In [104]:
a = np.array([1,2,3,4])
a + a.reshape(-1,1)*10

array([[11, 12, 13, 14],
       [21, 22, 23, 24],
       [31, 32, 33, 34],
       [41, 42, 43, 44]])

In [116]:
BaumWelch_init = namedtuple('BaumWelch_init', 'Mu, A, B')
BaumWelch_result = namedtuple('BaumWelch_result', 'Mu, A, B, prob, iters')

# X, Y: T
# mu: 1xN
# A: NxN
# B: NxM
def learn(Y:list[int], params:BaumWelch_params, init_vals=(None, None, None), seed=42):
    np.random.seed(seed)
    N, M, T = params.n_hidden_states, params.n_observation_states, params.n_observations
    
    Mu, A, B = init_vals
    if Mu is None: 
        Mu = np.ones(N) + (np.random.rand(N) - 0.5)*0.001
        Mu = Mu / Mu.sum()
    if A is None: 
        A = np.ones((N,N)) + (np.random.rand(N,N) - 0.5)*0.001
        A = A / A.sum(axis=1, keepdims=True)
    if B is None: 
        B = np.ones((N,M)) + (np.random.rand(N,M) - 0.5)*0.001
        B = B / B.sum(axis=1, keepdims=True)

    if (np.abs(Mu.sum() - 1) > STOCHASTIC_EPS).any(): raise ValueError(f"Initial Mu is not stochastic! s={Mu.sum()}")
    if (np.abs(A.sum(axis=1) - 1) > STOCHASTIC_EPS).any(): raise ValueError(f"Initial A is not stochastic! s={A.sum(axis=1)}")
    if (np.abs(B.sum(axis=1) - 1) > STOCHASTIC_EPS).any(): raise ValueError(f"Initial B is not stochastic! s={B.sum(axis=1)}")


    alpha, c = calculate_alpha(Y, Mu, A, B, params)
    print("init c:", c)
    prob = evaluate(c)
    print(f"Starting Baum-Welch process. Initial probability = {prob}")
    print("Initial Mu:", Mu)
    print("Initial A:", A)
    print("Initial B:", B)
    counter = 0
    while True:
        counter += 1
        newMu, newA, newB = learn_iterate(Y, Mu, A, B, params, alpha, c)

        if (np.abs(newMu.sum() - 1) > STOCHASTIC_EPS).any(): raise ValueError(f"New Mu is not stochastic! iter = {counter}, s={newMu.sum()}")
        if (np.abs(newA.sum(axis=1) - 1) > STOCHASTIC_EPS).any(): raise ValueError(f"New A is not stochastic! iter = {counter}, s={newA.sum(axis=1)}")
        if (np.abs(newB.sum(axis=1) - 1) > STOCHASTIC_EPS).any(): raise ValueError(f"New B is not stochastic! iter = {counter}, s={newB.sum(axis=1)}")

        alpha, c = calculate_alpha(Y, newMu, newA, newB, params)
        new_prob = evaluate(c)

        if new_prob - prob < params.eps:
            # intentionally no abs to handle new_prob < prob 
            print("Stopping on eps")
            break
        Mu, A, B = newMu, newA, newB
        prob = new_prob

        if counter > params.max_iter:
            print("Stopping on max_iter")
            break
        print(f"Iteration {counter} finished. p={prob}")

    return BaumWelch_result(Mu, A, B, prob, counter)


---

### 2.2. Виконання

In [106]:
# читання тексту
data = read_textfile(TEXTFILE, N_OBSERVATIONS)
print("Size read: ", len(data))
    

read checkpoint 10000 (11496)
read checkpoint 20000 (22649)
read checkpoint 30000 (33737)
read checkpoint 40000 (45201)
read checkpoint 50000 (56530)
read - end
Size read:  50001


In [120]:
# уривок зчитаного тексту
data[:20]

bytearray(b'\xef\xb3\xf2\xe5\xf0 \xe2\xee\xf2\xf2\xf1 \xec\xee\xf0\xf1\xfc\xea\xe0 ')

In [115]:
Y = np.empty(N_OBSERVATIONS, np.uint8)
for i,c in enumerate(data):
    if i >= N_OBSERVATIONS: continue
    Y[i] = LETTERS.find(c)
    if Y[i] < 0:
        print(f"ERROR: invalid character {c} (somehow) at index {i}")
        break

params = BaumWelch_params(
    max_iter=100, eps=1e-7, 
    n_hidden_states=2, n_observation_states=len(LETTERS), n_observations=N_OBSERVATIONS
)
init_vals = BaumWelch_init(
    Mu=None, 
    A=np.array([[0.4, 0.6],[0.6, 0.4]]), 
    B=None
)
print(Y)
res = learn(Y, params, init_vals)
print(res)


[21  2 24 ... 14  8 11]
init c: [35.00898624 35.00575306 35.00794396 ... 34.99729613 35.00423159
 35.00290208]
Starting Baum-Welch process. Initial probability = -177768.2130619583
Iteration 1 finished. p=-157253.3220101003
Iteration 2 finished. p=-157253.32198579278
Iteration 3 finished. p=-157253.32195472423
Iteration 4 finished. p=-157253.32191410864
Iteration 5 finished. p=-157253.32186013422
Iteration 6 finished. p=-157253.32178734106
Iteration 7 finished. p=-157253.32168806484
Iteration 8 finished. p=-157253.32155127774
Iteration 9 finished. p=-157253.32136103683
Iteration 10 finished. p=-157253.32109403692
Iteration 11 finished. p=-157253.32071608337
Iteration 12 finished. p=-157253.32017638942
Iteration 13 finished. p=-157253.31939912305
Iteration 14 finished. p=-157253.31827046216
Iteration 15 finished. p=-157253.3166182929
Iteration 16 finished. p=-157253.31418101906
Iteration 17 finished. p=-157253.3105592014
Iteration 18 finished. p=-157253.3051401211
Iteration 19 finished.

### 2.3. Результати

In [117]:
res.A

array([[0.37934349, 0.62065651],
       [0.98769021, 0.01230979]])

In [118]:
res.B

array([[2.15160917e-01, 1.20500970e-03, 2.52750902e-20, 2.93110467e-04,
        1.84659594e-02, 8.41827322e-03, 3.49537596e-24, 2.41978974e-02,
        6.50053879e-02, 2.12342249e-02, 4.60183418e-02, 1.71198315e-09,
        1.50137695e-02, 2.99213267e-02, 7.03307342e-38, 1.49812016e-02,
        5.22575094e-02, 5.50070642e-02, 4.13611436e-02, 8.74120547e-02,
        1.06182065e-04, 3.09365738e-02, 6.49076844e-02, 5.21910891e-02,
        7.69903492e-02, 5.90113179e-18, 2.44258722e-03, 1.24409109e-02,
        1.09102229e-02, 1.98989439e-02, 1.13618658e-02, 8.40250004e-03,
        1.15029075e-03, 1.16139724e-02, 6.93633773e-04],
       [6.14906153e-02, 1.12824668e-14, 1.09718380e-01, 8.08316680e-91,
        4.11074367e-34, 5.52041951e-03, 1.91294540e-01, 8.56439218e-30,
        5.02560452e-14, 1.51055703e-24, 2.32988733e-09, 1.06764222e-01,
        5.43963755e-14, 6.33647028e-03, 1.33818071e-01, 3.48086556e-14,
        1.25706406e-04, 1.06304551e-56, 1.38939013e-49, 2.89945802e-58,
       

In [119]:
# print(LETTERS, type(LETTERS))
utf8letters = codecs.iterdecode([LETTERS], 'cp1251').__next__()
labels = list(np.argmax(res.B, axis=0))
for c in utf8letters:
    print(f"{c} ", end='')
print()
for l in labels:
    print(f"{l} ", end='')
print()

  ' і ґ є ї а б в г д е ж з и й к л м н о п р с т у ф х ц ч ш щ ь ю я 
0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 


Маємо дійсно прекрасний результат. Алгоритм Баума-Велша навчив ПММ таким чином, що з самих лише даних вдалося виділити голосні та приголосні літери (якщо не зважати на знаки). Що цікаво, до класу приголосних також віднесено йотовані голосні, що узгоджується з правилами української мови.

## 3. Більше кластерів! N=3

### 3.1. Код

In [127]:
Y = np.empty(N_OBSERVATIONS, np.uint8)
for i,c in enumerate(data):
    if i >= N_OBSERVATIONS: continue
    Y[i] = LETTERS.find(c)
    if Y[i] < 0:
        print(f"ERROR: invalid character {c} (somehow) at index {i}")
        break

params = BaumWelch_params(
    max_iter=300, eps=1e-7, 
    n_hidden_states=3, n_observation_states=len(LETTERS), n_observations=N_OBSERVATIONS
)
init_vals = BaumWelch_init(
    Mu=None, 
    A=np.array([[0.4, 0.3, 0.3],[0.3, 0.4, 0.3],[0.3, 0.3, 0.4]]), 
    B=None
)
print(Y)
res = learn(Y, params, init_vals)
print(res)


[21  2 24 ... 14  8 11]
init c: [34.99814766 35.00451005 34.99560266 ... 34.99173857 34.99403484
 35.00835437]
Starting Baum-Welch process. Initial probability = -177766.4216821426
Initial Mu: [0.33322962 0.33342164 0.33334875]
Initial A: [[0.4 0.3 0.3]
 [0.3 0.4 0.3]
 [0.3 0.3 0.4]]
Initial B: [[0.02857623 0.02856358 0.02856358 0.02856079 0.02858388 0.0285763
  0.02857936 0.02855971 0.02858684 0.02858291 0.02856519 0.02856432
  0.02856437 0.02856782 0.02857412 0.02857147 0.02856745 0.02857661
  0.02856311 0.02856747 0.02856959 0.02857216 0.02858156 0.02856483
  0.02857382 0.02857605 0.02856045 0.02857649 0.028564   0.02856099
  0.02858624 0.02858672 0.02858223 0.02856783 0.02856192]
 [0.02857745 0.02857047 0.02856139 0.02857205 0.02855888 0.02858388
  0.02856529 0.02857683 0.0285668  0.02857276 0.02857352 0.02856318
  0.0285856  0.02858005 0.02858474 0.02858347 0.02857498 0.02858424
  0.02856043 0.0285635  0.02855919 0.02856719 0.028569   0.02856565
  0.02858158 0.02856809 0.02856593 

In [128]:
res.A

array([[1.73311451e-07, 5.12784350e-01, 4.87215477e-01],
       [1.88546183e-04, 2.82194864e-01, 7.17616590e-01],
       [8.96632157e-01, 5.06075514e-04, 1.02861767e-01]])

In [129]:
res.B

array([[1.67356212e-002, 2.09439097e-003, 1.19754374e-001,
        4.89369850e-079, 1.86702760e-026, 6.95203255e-045,
        2.06160706e-001, 1.31351484e-055, 3.01715518e-053,
        1.13509408e-044, 1.96542563e-003, 1.13785574e-001,
        2.84799488e-005, 2.30126402e-031, 1.46154526e-001,
        3.91995739e-062, 3.89976781e-003, 5.16662609e-028,
        5.37076686e-083, 1.05625216e-013, 2.05036845e-001,
        2.90859531e-015, 5.61776178e-034, 1.33035853e-016,
        1.01233445e-007, 8.03745673e-002, 5.25639720e-074,
        5.93577049e-052, 4.41206406e-057, 9.18504375e-045,
        7.79841965e-020, 7.63591507e-110, 4.91898851e-002,
        8.45772981e-003, 4.63620054e-002],
       [5.25634348e-001, 4.84286079e-074, 1.10124080e-004,
        1.83970641e-122, 4.48805005e-002, 2.88913275e-002,
        3.87197686e-003, 8.26687886e-003, 3.58854597e-002,
        6.23073196e-005, 3.35247067e-002, 3.94485138e-003,
        7.26838184e-003, 3.44730626e-002, 1.81894190e-047,
        3.266

### 3.2. Результати

- 100 ітерацій не дали нічого
- 300 ітерацій змогли виділити клас голосних літер (із апострофом та м'яким знаком), але інші класи я не зрозумів

In [130]:
# print(LETTERS, type(LETTERS))
utf8letters = codecs.iterdecode([LETTERS], 'cp1251').__next__()
labels = list(np.argmax(res.B, axis=0))
for c in utf8letters:
    print(f"{c} ", end='')
print()
for l in labels:
    print(f"{l} ", end='')
print()

  ' і ґ є ї а б в г д е ж з и й к л м н о п р с т у ф х ц ч ш щ ь ю я 
1 0 0 2 1 1 0 2 2 2 2 0 2 1 0 1 2 2 2 2 0 2 2 1 2 0 2 1 2 2 2 2 0 1 0 
